# Projeto 2 - Ciência dos Dados

Nome: Carlos Eduardo Dip

Nome: Gianluca Lazzaris Giudici

# Proposta
___

Neste projeto, a proposta é criar um identificador de sentimento, a partir de um analisador Naïve-Bayes, que faz uso principalmente do Teorema de Bayes, e é um modelo de Machine Learning.
___
# Classificador automático de sentimento

<img src = 'logo.jpg' style = 'width:30%'>

Empresa escolhida: Uber

___
## Preparando o ambiente no jupyter:

Primeiro importamos os módulos que serão utilizados:

In [61]:
import numpy as np
import os.path
import pandas as pd
import json
import re
import string
import functools
import operator
from random import shuffle

try:
    import emoji
except:
    !pip install emoji --upgrade
    import emoji

___
### Coleta de Dados:

Para a análise que será feita, serão usados tweets (da plataforma Twitter), que serão extraídos através de uma API chamada Tweepy;
Os dados foram coletados em outro notebook, chamado Coletor.ipynb. Abaixo, será importado o excel com os dados já classificados.

___

## Limpeza, aquisição e organização dos dados

Primeiro, construímos dataframes (da biblioteca Pandas), para facilitar a visualização e utilização dos dados.

In [62]:
data_training = pd.read_excel('data/Uber.xlsx')
data_validation = pd.read_excel('data/Uber.xlsx', 'Teste')

data_training_R = data_training.loc[data_training.Classificacao == 1]
data_training_NR = data_training.loc[data_training.Classificacao == 0]
TrainingString_Relevant = ''
TrainingString_NotRelevant = ''

Agora, podemos visualizar o formato que o dataframe se encontra. Ele possuí 2 colunas (fora o índice), uma delas indica qual a classificação (1 é relevante, e 0 é irrelevante), e a outra mostra o tweet que foi avaliado.


In [63]:
data_training.head(10)

,Treinamento,Classificacao
0,uma corrida com esse uber a única forma de pag...,0
1,uber testa em são paulo programa que avalia qu...,1
2,"vcs que andam de uber nem falem comigo, além d...",0
3,a fic: terminei o último episódio de glee (dnv...,1
4,"olivares, cida \nsobre seu uber \n\nmas podia ...",0
5,"nossa que vontade de sentar no uber, quer dize...",0
6,adoro ler os feedbacks do uber https://t.co/lp...,1
7,uber com balinha tudo pra mim,1
8,so uber e faço isso todo dia... https://t.co/e...,1
9,moço do uber: vc é a melhor da classe? \neu: n...,1


In [64]:
data_training.Classificacao.value_counts(True)

1    0.56
0    0.44
Name: Classificacao, dtype: float64

Com a série acima, percebemos que 56% dos tweets são relevantes, e 44% não, de acordo com nossa avaliação manual.

Podemos perceber, contudo, que esses tweets possuem pontuação, emojis, e alguns caractéres como **\n**, que irão poluir a análise.

Por isso, produzimos algumas funções que são capazes de limpar os tweets, para facilitar a análise.

In [65]:
def check_laughter(word):
    for letter in word:
        if letter != 'k':
            return False
    return True

def cleanup(text):

    text_split_emoji = emoji.get_emoji_regexp().split(text)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    text_split = ' '.join(word for word in text_split)
    
    text = ' '.join(word for word in text_split.split() if not word.startswith('https'))
    
    
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = text_subbed.lower()
    text_out = ' '
    
    
    for word in text_subbed.split():
        if check_laughter(word):
            text_out += ' haha'
        else:
            text_out += ' ' + word
    
    
    return text_out    

In [66]:
### --- Cleanup function demo --- ###
demo_string_ =  """
                X gon give it to ya
                Fuck wait for you to get it on your own
                X gon deliver to ya
                Knock knock, open up the door, it's real
                Wit the non-stop, pop pop and stainless steel
                Go hard gettin busy wit it
                https://img.imgur/1337, uber blah blah kkk, 👨🏿😷😷
                https://img.imgur/1337, uber blah blah kkkkkkk, 👨🏿😷😷
                """

print(cleanup(demo_string_))
print(cleanup(demo_string_).split())

  x gon give it to ya fuck wait for you to get it on your own x gon deliver to ya knock knock open up the door it s real wit the non stop pop pop and stainless steel go hard gettin busy wit it uber blah blah haha 👨🏿 😷 😷 uber blah blah haha 👨🏿 😷 😷
['x', 'gon', 'give', 'it', 'to', 'ya', 'fuck', 'wait', 'for', 'you', 'to', 'get', 'it', 'on', 'your', 'own', 'x', 'gon', 'deliver', 'to', 'ya', 'knock', 'knock', 'open', 'up', 'the', 'door', 'it', 's', 'real', 'wit', 'the', 'non', 'stop', 'pop', 'pop', 'and', 'stainless', 'steel', 'go', 'hard', 'gettin', 'busy', 'wit', 'it', 'uber', 'blah', 'blah', 'haha', '👨🏿', '😷', '😷', 'uber', 'blah', 'blah', 'haha', '👨🏿', '😷', '😷']


Acima, temos um exemplo de uso da função que limpa os textos necessários. Ela é capaz de remover **\n** (caractéres de quebra de linha), hyperlinks, pontuações, e também separa emojis das palavras e entre si. Outra coisa que será útil para a análise é transformar risadas (comumente representadas por uma série de caractéres *k* seguidos), em uma representação padrão, que foi escolhida como 'haha'.

In [67]:
for phrase in data_training_R.Treinamento:
    TrainingString_Relevant += phrase
for phrase in data_training_NR.Treinamento:
    TrainingString_NotRelevant += phrase    
    
cleanup(TrainingString_NotRelevant)
cleanup(TrainingString_Relevant)

TrainingSeries_Relevant = pd.Series(TrainingString_Relevant.split()).value_counts(True)
TrainingSeries_NotRelevant = pd.Series(TrainingString_NotRelevant.split()).value_counts(True)

LaplaceConstant = 1

SampleSize_Relevant = len(TrainingString_Relevant.split())
SampleSize_NotRelevant = len(TrainingString_NotRelevant.split())
SampleSize_Total = SampleSize_Relevant+SampleSize_NotRelevant


___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [68]:
P_Relevant = SampleSize_Relevant/SampleSize_Total
P_NotRelevant = SampleSize_NotRelevant/SampleSize_Total
print(P_NotRelevant, P_Relevant)

0.44193345888261143 0.5580665411173886


In [69]:
def evaluate_relevance(tweet = ""):
    
    ## -- Cleaning tweet
    text = cleanup(tweet)
    
    ## -- Evaluates likelihood
    sumR = 0
    sumNR = 0
    
    for word in text.split():
    
        if word in TrainingSeries_Relevant: 
            sumR += np.log((TrainingSeries_Relevant[word]+LaplaceConstant)/((SampleSize_Total+1)*LaplaceConstant))
        else:
            sumR += np.log(LaplaceConstant/((SampleSize_Total+1)*LaplaceConstant))
        if word in TrainingSeries_NotRelevant: 
            sumNR += np.log((TrainingSeries_NotRelevant[word]+LaplaceConstant)/((SampleSize_Total+1)*LaplaceConstant))
        else:
            sumNR += np.log(LaplaceConstant/((SampleSize_Total+1)*LaplaceConstant))
     
    
    return sumR>sumNR


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [70]:
evaluations = [int(evaluate_relevance(tweet)) for tweet in data_validation.Teste]

In [71]:
data_validation['bayes'] = evaluations

In [72]:
Accuracy = {'True-True':0,
            'True-False':0,
            'False-True':0,
            'False-False':0
           }


for tweet,evaluation in zip(data_validation.classificacao, data_validation.bayes):
    if tweet and evaluation:
        Accuracy['True-True'] += 1
    elif tweet and not evaluation:
        Accuracy['True-False'] += 1
    elif not tweet and evaluation:
        Accuracy['False-True'] += 1
    elif not tweet and not evaluation:
        Accuracy['False-False'] += 1

Accuracy_Normalized = {}
S = sum(Accuracy.values())
for k,v in Accuracy.items():
    Accuracy_Normalized[k] = v/S
Accuracy_Normalized['Final-Accuracy'] = str(round((Accuracy_Normalized['True-True']+Accuracy_Normalized['False-False'])*100, 3)) + '%'
Accuracy_Normalized

{'True-True': 0.36180904522613067,
 'True-False': 0.16080402010050251,
 'False-True': 0.3165829145728643,
 'False-False': 0.16080402010050251,
 'Final-Accuracy': '52.261%'}

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) **Explicação da técnica para lidar com palavras novas**